In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/welfake-dataset/WELFake_Dataset.csv


In [19]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import nltk
import re
import string
from keras.layers import Dense, Embedding, LSTM, Bidirectional
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
df = pd.read_csv('/kaggle/input/welfake-dataset/WELFake_Dataset.csv')
df['text'] = df['title'] + " " + df['text']
df = df[['text', 'label']]
df = df[df['text'].notna()]
df = df.reset_index(drop=True)

In [ ]:
new_df=df.copy()

In [ ]:
def review_cleaning(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
new_df['text']=new_df['text'].apply(lambda x:review_cleaning(x))

In [ ]:
stop = stopwords.words('english')
new_df['text'] = new_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [22]:
import torch

is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
print(device)

cuda


In [23]:
sentences = new_df['text'].values
print(len(sentences))

72095


In [24]:
from transformers import BertForSequenceClassification, BertTokenizer, RobertaForSequenceClassification, RobertaTokenizer, AdamW

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [25]:
model_bert = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2,output_attentions=False,output_hidden_states=False)
# 'bert-base-uncased' : 12 layer BERT model with uncased vocab
# num_labels : 2 labels for binary classification
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert.cuda()

print("\n\nBERT Model : \n\n",model_bert)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at



BERT Model : 

 BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(

In [26]:
model_roberta = RobertaForSequenceClassification.from_pretrained('roberta-base',num_labels=2,output_attentions=False,output_hidden_states=False)
# 'roberta-base' : 12 layer, 768 hidden, 12 heads, 125M params RoBERTa using BERT-base architecture
tokenizer_roberta = RobertaTokenizer.from_pretrained('roberta-base')
model_roberta.cuda()

print("\n\nRoBERTa Model : \n\n",model_roberta)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie



RoBERTa Model : 

 RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bia

In [27]:
### Tokenization : encode_plus method from tokenizer_bert and tokenizer_roberta

# encode plus : tokenize sentence, prepand [CLS] to start, append [SEP] to end, 
# map token to their ID, Pad or truncate the sentence to max_len, create attention masks for [PAD] tokens
final_labels = np.array(new_df['label'])
inputID_bert = []
attentionMask_bert = []

inputID_roberta = []
attentionMask_roberta = []

sentenceID = []
count = 0

for text in sentences:
    
    enc_dict_bert = tokenizer_bert.encode_plus(text,add_special_tokens=True,max_length=120,pad_to_max_length=True, return_attention_mask=True,return_tensors='pt')
    enc_dict_roberta = tokenizer_roberta.encode_plus(text,add_special_tokens=True,max_length=120,pad_to_max_length=True, return_attention_mask=True,return_tensors='pt')
    
    # max_length : Pad and truncate all texts
    # return_attention_mask : construct attention masks
    # return_tensors : 'pt' : pytorch tensor
    
    inputID_bert.append(enc_dict_bert['input_ids'])
    inputID_roberta.append(enc_dict_roberta['input_ids']) # added encoded text as ID to the list
    
    attentionMask_bert.append(enc_dict_bert['attention_mask']) # added attention mask to the list
    attentionMask_roberta.append(enc_dict_roberta['attention_mask']) # that simply differs padding from non-padding

    sentenceID.append(count)
    count = count + 1
    
# convert lists to tensor

inputID_bert = torch.cat(inputID_bert,dim=0)
inputID_roberta = torch.cat(inputID_roberta,dim=0)
attentionMask_bert = torch.cat(attentionMask_bert,dim=0)
attentionMask_roberta = torch.cat(attentionMask_roberta,dim=0)

labels = torch.tensor(final_labels)
sentenceID = torch.tensor(sentenceID)

print('\nOriginal: \n', sentences[0])
print('\nToken IDs BERT: \n', inputID_bert[0])
print('\nToken IDs RoBERTa: \n', inputID_roberta[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `


Original: 
 No comment is expected from Barack Obama Members of the #FYF911 or #FukYoFlag and #BlackLivesMatter movements called for the lynching and hanging of white people and cops. They encouraged others on a radio show Tuesday night to  turn the tide  and kill white people and cops to send a message about the killing of black people in America.One of the F***YoFlag organizers is called  Sunshine.  She has a radio blog show hosted from Texas called,  Sunshine s F***ing Opinion Radio Show. A snapshot of her #FYF911 @LOLatWhiteFear Twitter page at 9:53 p.m. shows that she was urging supporters to  Call now!! #fyf911 tonight we continue to dismantle the illusion of white Below is a SNAPSHOT Twitter Radio Call Invite   #FYF911The radio show aired at 10:00 p.m. eastern standard time.During the show, callers clearly call for  lynching  and  killing  of white people.A 2:39 minute clip from the radio show can be heard here. It was provided to Breitbart Texas by someone who would like to be

In [29]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [30]:
def sid_remove_from_tensordataset(datatensor): 
    
    inputID = []
    attentionMask = []
    label = []
    
    for sid,iid,amask,l in datatensor:
        inputID.append(iid.tolist())
        attentionMask.append(amask.tolist())
        label.append(l.tolist())
    
    inputID = torch.tensor(inputID)
    attentionMask = torch.tensor(attentionMask)
    label = torch.tensor(label)
    
    return TensorDataset(inputID,attentionMask,label)
    
# Get DataSetLoaders
def get_loaders(dataset,batch_size,b):

    """
    return the train, validation and test set loaders
    """  
    #dataset = torch.utils.data.TensorDataset(data_tr, labels_tr)
    train_size = int(0.8 * len(dataset))
    val_size = int(0.1 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    #print("\nTrain DataSet Size :",train_size)
    #print("\nValidation DataSet Size :",val_size)
    #print("\nTest DataSet Size :",test_size)
    train_dataset, validation_dataset,test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])
    
    if(b==1): # sid remove only in BERT model : b=1
        train_dataset = sid_remove_from_tensordataset(train_dataset)
        validation_dataset = sid_remove_from_tensordataset(validation_dataset)
        test_dataset = sid_remove_from_tensordataset(test_dataset)

    train_loader = DataLoader(train_dataset, batch_size=batch_size,sampler=RandomSampler(train_dataset))
    valid_loader = DataLoader(validation_dataset, batch_size=batch_size,sampler=SequentialSampler(validation_dataset))    
    test_loader = DataLoader(test_dataset, batch_size=batch_size,sampler=SequentialSampler(test_dataset))

    return train_loader, valid_loader, test_loader

In [31]:
def get_accuracy(y_pred, y_test):
    y_pred_flat = np.argmax(y_pred, axis=1).flatten()
    y_test_flat = y_test.flatten()
    return np.sum(y_pred_flat == y_test_flat) / len(y_test_flat)

In [32]:
def train_model(model, optimizer, train_loader):
  
    model.train()

    epoch_loss = 0
    epoch_acc = 0

    for iid, amask, labels in train_loader:
        
        iid, amask, labels = iid.to(device), amask.to(device), labels.to(device)
        model.zero_grad()
        loss,outputs = model(iid,token_type_ids=None, attention_mask=amask,labels=labels, return_dict=False)  
        #torch.set_default_tensor_type(torch.FloatTensor)
        
        #x = torch.tensor(x, dtype=torch.float32)
        #labels=torch.tensor(labels, dtype= torch.float32)
        #loss = criterion(outputs, labels)
        #loss = criterion(outputs.squeeze(), labels.float32)
        epoch_loss += loss.item()
        loss.backward()
        #nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        epoch_acc += get_accuracy(outputs.detach().cpu().numpy(),labels.to('cpu').numpy())

    train_loss =  epoch_loss / len(train_loader)
    train_acc = epoch_acc / len(train_loader)  
    return train_loss, train_acc

In [33]:
def evaluate_model(model, loader):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    y_true=[]
    y_pred=[]
    
    with torch.no_grad():
    
        for iid, amask, labels in loader:
            iid, amask, labels = iid.to(device), amask.to(device), labels.to(device)

            loss,outputs = model(iid,token_type_ids=None, attention_mask=amask, labels=labels, return_dict=False)
            #torch.set_default_tensor_type(torch.FloatTensor)
            labels=torch.tensor(labels, dtype= torch.float32)
            
            #e_loss = criterion(outputs.squeeze(), labels.float32)
            #e_loss = criterion(outputs,labels)
            epoch_loss += loss.item()
            
            epoch_acc += get_accuracy(outputs.detach().cpu().numpy(),labels.to('cpu').numpy())
           
            y_true.append(labels.to('cpu').numpy())
            y_pred.append(outputs.detach().cpu().numpy())
            
        
    loss =  epoch_loss / len(loader)
    acc = epoch_acc / len(loader)  
    return loss, acc, y_pred, y_true

In [34]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
def run_model(model, train_loader, validate_loader, test_loader, epochs, batch_size, optimizer):
   
    for epoch in range(epochs):
        train_loss, train_acc = train_model(model, optimizer, train_loader)
        valid_loss, valid_acc, _, _ = evaluate_model(model, validate_loader)

        print(f'Epoch: {epoch+1:02}')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

    ##Evaluate the test accuracy

    test_loss, test_acc, y_pred, y_true = evaluate_model(model, test_loader)
    print(f'\nTest Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
    
    flat_pred = np.concatenate(y_pred,axis=0)
    flat_pred = np.argmax(flat_pred,axis=1).flatten()
    flat_true = np.concatenate(y_true, axis=0)
    print("\nConfusion Matrix : \n",confusion_matrix(flat_true,flat_pred))
    print("\nClassification Report : \n", classification_report(flat_true,flat_pred))
    
NUM_EPOCHS = 1
batch_size = 20

In [23]:
# RUN BERT Model
print("\n\nBERT Model :\n")
dataset_bert = TensorDataset(sentenceID, inputID_bert, attentionMask_bert, labels)
train_loader_bert, validation_loader_bert, test_loader_bert = get_loaders(dataset_bert, batch_size, 1)
optimizer_bert = AdamW(model_bert.parameters(),lr=5e-5,eps=1e-8)

run_model(model_bert, train_loader_bert, validation_loader_bert, test_loader_bert, NUM_EPOCHS, batch_size, optimizer_bert)



BERT Model :



/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/tmp/ipykernel_31/1125999774.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels=torch.tensor(labels, dtype= torch.float32)


Epoch: 01
	Train Loss: 0.034 | Train Acc: 98.85%
	 Val. Loss: 0.030 |  Val. Acc: 98.95%

Test Loss: 0.028 | Test Acc: 99.04%

Confusion Matrix : 
 [[3468   55]
 [  14 3673]]

Classification Report : 
               precision    recall  f1-score   support

         0.0       1.00      0.98      0.99      3523
         1.0       0.99      1.00      0.99      3687

    accuracy                           0.99      7210
   macro avg       0.99      0.99      0.99      7210
weighted avg       0.99      0.99      0.99      7210



In [35]:
# RUN RoBERTa Model
print("\n\nRoBERTa Model :\n")
dataset_roberta = TensorDataset(inputID_roberta, attentionMask_roberta, labels)
train_loader_roberta, validation_loader_roberta, test_loader_roberta = get_loaders(dataset_roberta, batch_size, 0)
optimizer_roberta = AdamW(model_roberta.parameters(),lr=5e-5,eps=1e-8)

run_model(model_roberta, train_loader_roberta, validation_loader_roberta, test_loader_roberta, NUM_EPOCHS, batch_size, optimizer_roberta)



RoBERTa Model :



/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/tmp/ipykernel_31/1125999774.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels=torch.tensor(labels, dtype= torch.float32)


Epoch: 01
	Train Loss: 0.039 | Train Acc: 98.70%
	 Val. Loss: 0.016 |  Val. Acc: 99.61%

Test Loss: 0.012 | Test Acc: 99.64%

Confusion Matrix : 
 [[3553    8]
 [  18 3631]]

Classification Report : 
               precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      3561
         1.0       1.00      1.00      1.00      3649

    accuracy                           1.00      7210
   macro avg       1.00      1.00      1.00      7210
weighted avg       1.00      1.00      1.00      7210

